In [ ]:
from AnthropicWrapper import ClaudeChatCV

In [ ]:
model = "claude-3-5-sonnet-20240620"
pdf_path = "D:\Hidden Desktop\OneDrive\Cross Device\Jobs Applications\Graduating\CV.pdf"

In [ ]:
job_role = "Machine Learning Engineer"
candidate_skill = "Entry-Level"

role_description = """
Contract Type : Permanent

Wellcome is a global charitable foundation. We improve health for everyone by funding science, leading policy and advocacy campaigns, and building partnerships.

We plan to spend £16bn over the next ten years, funding new discoveries in life, health and wellbeing, and taking on three global health challenges: mental health, infectious disease and climate.

These challenges need the bold research our funding supports, but they won’t be solved by science alone. We are looking for a Machine Learning Engineer to join our growing Machine Learning team.

In this role, you will work together with an interdisciplinary team of software engineers, data engineers, analysts, and designers to help Wellcome gain a deeper understanding of its funding data and inform decision-making.

You will have the opportunity to develop machine learning models understand our research funding and it's impacts. With an emphasis on natural language processing and network analysis. You will work with open-source software in a highly collaborative team, developing, maintaining, and proposing new solutions to enhance our data science offering to the organisation

In this role you will:

Propose, develop, and evaluate machine learning models with an emphasis on natural language processing
Translate business problems to data science tasks and present complex results to stakeholders
Collaborate on a cross-functional team, sharing ideas, doing code reviews, maintaining, and improving our internal data science tooling
Identify opportunities where machine learning can help Wellcome make better decisions Contribute to a diverse and inclusive culture across the organisation, collaborating across departments

Is this job for me?

To be successful in this role you will have:

Good knowledge of Python and SQL.
Good experience with core data science libraries such as: Scikit-Learn, Pandas, PyTorch/TensorFlow, Hugging Face.
Knowledge of core machine learning and natural language processing techniques such as supervised/unsupervised learning, text classification, text mining and topic modelling
Experience with version control systems such as git/GitHub, python package management, unit testing and code reviews
Experience working with data exploration and data visualisation packages, such as: Pandas, Matplotlib, Bokeh, Plotly, Tableau
Exposure to working in a cloud environment such as AWS.
Experience working collaboratively in a multidisciplinary team with other machine learning and software engineers.
"""

In [ ]:
system_prompt = f"""You are a skilled interviewer who is conducting an initial phone screening interview for a candidate for a {candidate_skill} {job_role} role to see if the candidate is at minimum somewhat qualified for the role and worth the time to be fully interviewed. The role and company description is copypasted from the job posting as follows: {role_description}. Parse through it to extract any information you feel is relevant.
Your job is to begin a friendly discussion with the candidate, and ask questions relevant to the {job_role} role, which may or may not be based on the interviewee's CV, which you have access to. Be sure to stick to this topic even if the candidate tries to steer the conversation elsewhere. If the candidate has other experience on his CV, you can ask about it, but keep it within the context of the {job_role} role.
After the candidate responds to each of your questions, you should not summarise or provide feedback on their responses. THIS POINT IS KEY! You should not summarise or provide feedback on their responses. You must keep your responses short and concise without reiterating what is good about the candidate's response or experience when they reply.
You can ask follow-up questions if you wish.
Once you have asked sufficient questions such that you deem the candidate is or isn't fitting for the role, end the interview by thanking the candidate for their time and informing them that they will receive word soon on the outcome of the screening interview. If the candidate does not seem fititng for the role, or if something feels off such as the candidate being unconfident or very very vague feel free to end the interview early. There is no need to inform them of your opinion of their performance, as this will be evaluated later.
The candidate will begin the interview by greeting you. You are to greet them back, and begin the interview."""

In [ ]:
claude = ClaudeChatCV(model, system_prompt, pdf_path)

In [ ]:
while True:
    print(claude.chat_with_history_doc(input()))

In [ ]:
#del(claude.message_history[-1])

In [9]:
conversation = claude.get_conversation()
print(conversation)

role: user
content: The following is a CV that I am providing you with. You are to keep this document in the back of your mind and consider it or use it, should it be relevant to the discussion.

 Document below: 



## Niranjan Kewalramani


## About Me


## Dedicated and creative machine learning engineer from the UK with a focus on building Machine Learning solutions, 
especially incorporating Large Language Models. Graduating with a master’s degree in Machine Learning (predicted 
Distinction) with key ML skills with a strong drive for continuous learning. Proficient in Python, Java and C++, as well 
as several ML frameworks such as TensorFlow Keras and PyTorch, and capable of rapidly learning anything else 
needed to succeed.


## Currently, developing projects and ideas into market-ready solutions part-time, with one set to launch in the coming 
months. Competitor in the Google Gemini API Developer competition, and frequent participant in Kaggle 
competitions, constantly improving

In [15]:
import re
from fpdf import FPDF
# Function to extract role and content from each turn
def parse_turn(turn):
    role_match = re.search(r"role: (.*)", turn)
    content_match = re.search(r"content: (.*)", turn, re.DOTALL)

    role = role_match.group(1) if role_match else ""
    content = content_match.group(1).strip() if content_match else turn.strip()

    return role, content

# Create a list of dictionaries, each representing a turn in the conversation
conversation_list = []
for turn in conversation.split("--------------------\n"):
    if turn.strip():  # Ignore empty turns
        role, content = parse_turn(turn)
        #print(content)
        if "Understood! I'll keep this CV in the back of my mind and use it should it be relevant to the discussion." not in content and "The following is a CV that I am providing you with." not in content:
            conversation_list.append({"role": role, "content": content})

# Create a PDF object
pdf = FPDF()
pdf.add_page()

# Set margins (in millimeters)
pdf.set_margins(left=10, top=20, right=10)  # Right margin set to 10mm 

# Choose a nicer font
pdf.set_font("Helvetica", size=12)

# Calculate usable width for text wrapping (accounting for margins)
usable_width = pdf.w - pdf.l_margin - pdf.r_margin -0 # 10px right margin 

# Add conversation to the PDF
for turn in conversation_list:
    # Skip turns with empty roles or content
    if turn['role'] and turn['content']:
        # Subheading style for "User" and "Assistant"
        pdf.set_font("Helvetica", style="B", size=14)
        pdf.cell(0, 10, txt=f"{turn['role'].capitalize()}:", ln=True)

        # Content with regular font and correct indentation
        pdf.set_font("Helvetica", size=12)
        pdf.x = pdf.l_margin  # Reset x-coordinate to the left margin
        pdf.multi_cell(usable_width, 6, txt=turn['content'])
        pdf.ln(3)

# Save the PDF
pdf.output("conversation.pdf")

''